## 🐍Python Tricky Behaviour with Default Mutable Arguments - A big source of bugs if you dont know how to handle it🐍

❓ Try understanding the output of the code snippet ❓

💡DETAIL ANSWER BELOW💡

==========================

In [ ]:
def danger_function(numbers=[]):
    numbers.append(10)
    return numbers

call_1 = danger_function()
call_2 = danger_function([1,2,3,5])
call_3 = danger_function()

print(call_1, call_2)
#  [10, 10] [1, 2, 3, 5, 10]
# ❓EXPLAIN WHY ❓

### 🐍 The key concepts here are about this Core Principle in Python - That default mutable arguments (like lists) are shared between successive function calls if they're not provided by the caller. 

This is because they're initialized once when the function is defined, not every time the function is called. The default object (in this case, the list) is created once and stored in memory. 

🐍 The default argument's memory location doesn't change with each function call unless a new object is assigned to it. When the function is defined and a mutable default argument is provided, Python creates that mutable object and remembers it.

👉 Every time you call the function without providing that argument, the function uses the same object from memory. So if you modify that mutable object (like appending to the list), you're modifying the same object in memory.

👉🔥 This can lead to unexpected results and hard-to-diagnose bugs. Imagine a situation where this function is being called in different parts of a complex application. You'd expect each call to start with a fresh list, but due to the mutable default argument, it keeps accumulating key-value pairs from previous calls.🔥

In my opinion, this behavior of Python, kind of goes against the idea of functional programming. It makes it more difficult to write a deterministic function whose return value depends only on its arguments.

More fundamentally, it violates the principle of least surprise. This behavior isn’t what most programmers expect, which leads to bugs.

Let's dive in

---------------------

👉 When the `danger_function` is defined, the default argument for `numbers` is an empty list (`[]`). 

👉 When `danger_function()` is called the first time (i.e., `call_1 = danger_function()`), no argument is provided for `numbers`. Hence, the default empty list (`[]`) is used. The number 10 is then appended to this list.

👉 The second call to `danger_function` provides a list explicitly (`call_2 = danger_function([1,2,3,5])`). This means the function uses this provided list, rather than the default one. It appends 10 to the list, resulting in `[1, 2, 3, 5, 10]`. 

👉🐍 NOTE AGAIN - When you call the function with an argument (i.e. `call_2 = danger_function([1,2,3,5])`), it uses the list you provided, leaving the default list untouched during that call.

👉🐍 Now, here's where it gets interesting. For the third call, `call_3 = danger_function()`, no list argument is provided, so the function falls back to the default list. 🐍

But remember, the default list already had one `10` appended to it from the first call. So, the function appends another `10` to the same list, making it `[10, 10]`.

👉 The created list, during the first call of `danger_function` - is a single object in memory, and every time the function is invoked without a new list provided, it goes back to this same object.

👉 Finally, when printing `call_1` and `call_2`, it results in the observed output.

 👉 `call_1` and `call_3` point to the same list (i.e., the default list), which is why `call_1` displays `[10, 10]` and not just `[10]` as one might initially expect. And `call_2`, which had its list provided explicitly, displays `[1, 2, 3, 5, 10]`.

This behavior can lead to unexpected results, as you've seen, especially when mutable default arguments like lists or dictionaries are used. It's a common Python pitfall.

To avoid this, a common practice is to use `None` as the default value and check for it in the function body, like so:

```python
def safe_function(numbers=None):
    if numbers is None:
        numbers = []
    numbers.append(10)
    return numbers
```

With this approach, you won't encounter the shared mutable default argument issue.

============================

❓ So one question you may ask - Here how come `call_2 = danger_function([1,2,3,5])` is producing a separate list altogether. Why this call is NOT using the same `numbers` variable from the call_1 ❓

That's because, 

👉 When you call `danger_function` with an argument, as in `call_2 = danger_function([1,2,3,5])`, you are explicitly passing a new list to the function. In this case, the function doesn't use the default `numbers` list. Instead, it uses the list you provided, `[1,2,3,5]`.

👉 When you provide an argument to a function, it will always take precedence over the default value. This behavior is not unique to mutable default arguments; it's just how function arguments work in Python.

👉 The list `[1,2,3,5]` that you passed is a completely separate object in memory from the default list associated with the function. When the function appends `10` to it, you get `[1,2,3,5,10]`, but this modification has no effect on the default list from the function definition.

👉 So, in essence:

- When you call the function without an argument (`call_1` and `call_3`), it uses the default mutable list, which is shared across these calls.
- When you call the function with an argument (`call_2`), it uses the list you provided, leaving the default list untouched during that call.

---------------------


## Another example of the behaviour of Default-Mutable-Argument that can cause difficult to detect bugs

Consider an application that manages teams. There's a function to add a player to a team. If the team isn't specified, the player is added to a default team.

```python
def add_to_team(player, team=[]):
    team.append(player)
    return team
```

Now, let's walk through the potential pitfalls:

👉 Initially, Alice joins without specifying a team, so she gets added to the default team:

```python
print(add_to_team("Alice"))  # ['Alice']
```

👉 Next, Bob joins, also without specifying a team:

```python
print(add_to_team("Bob"))  # ['Alice', 'Bob']
```

## 👉 The expectation might be that Bob gets added to a new default team, but instead, he's added to the same team as Alice due to the mutable default argument.

👉 Now, if we add David without specifying a team, the result is still affected by previous calls:

```python
print(add_to_team("David"))  # ['Alice', 'Bob', 'David']
```

👉 One might wrongly assume that the behavior of adding without specifying a team is independent of adding to a specific team since they're separate operations. But due to the default mutable argument, they're intertwined, leading to hard-to-detect issues, especially in large codebases.

To safely achieve the intended functionality, you'd avoid mutable default arguments:

```python
def safe_add_to_team(player, team=None):
    if team is None:
        team = []
    team.append(player)
    return team
```

Using this approach ensures that every player added without a specific team is added to a new default team, and specific team additions don't interfere with the default behavior.

---------------------

## Yet another example - Let's explore the use of a mutable `set` as a default argument.

Consider a function designed to track unique error messages in an application. Each time a new error is found, it's added to a set. If no set is provided, the function adds errors to a default set.

```python
def track_error(message, error_set=set()):
    error_set.add(message)
    return error_set
```

Let's break down potential unexpected behaviors with this approach:

👉 Imagine the application finds an error message "Error 404":

```python
print(track_error("Error 404"))  # {'Error 404'}
```

👉 Later on, the application finds a different error, "Error 500":

```python
print(track_error("Error 500"))  # {'Error 404', 'Error 500'}
```

Instead of a new set containing just "Error 500", the function returns a set with both "Error 404" and "Error 500" due to the mutable default argument.

👉 If a new error in the main application arises, "Error 403":

```python
print(track_error("Error 403"))  # {'Error 404', 'Error 500', 'Error 403'}
```

Even though "FeatureError 101" is tracked separately, it might seem as if this error is somehow influencing the main application's errors due to the mutable default argument.

To resolve this, we can follow the pattern we previously discussed:

```python
def safe_track_error(message, error_set=None):
    if error_set is None:
        error_set = set()
    error_set.add(message)
    return error_set
```

This approach ensures that each error added without a specific set is added to a new default set, and specific sets are kept separate from the default behavior.

In summary, mutable default arguments, regardless of the specific type (list, set, dictionary, etc.), can introduce unexpected behaviors and entanglements between independent function calls. It's best to avoid them to ensure more predictable and bug-free code.

------------------

Even if your function does not mutate an optional parameter, unwanted side effects can occur if your function exposes the mutable value to the caller. For example, if the function returns the mutable value, and the calling code modifies its state, subsequent calls to the function will see the modifications, even though they are not explicitly passed in. Even worse, completely unrelated code (perhaps inside different modules) can call the same function and thereby contaminate each other’s inner workings.

-------------------